In [1]:
# 더보기 계속 클릭하기
def clickMore(driver):
    import time 
    
    while True:
        try:
            more_button = driver.find_element_by_css_selector('#spinTopLayerCommentListMore')
            more_button.click()
            time.sleep(1)
        except:
            break

In [27]:
# 한 기사의 댓글 추출
def getComment(driver):
    
    import os
    import time
    import pandas as pd
    
    # 데이터를 담을 딕셔너리
    comment_dict = {
        '제목' : [],
        '날짜' : [],
        '작성자' : [],
        '작성일' : [],
        '댓글' : []
    }
    
    # 기사 제목 태그
    a0 = driver.find_elements_by_css_selector('#container > div.article_title')[0]
    
    # 기사 제목
    data1 = a0.find_elements_by_css_selector('h1')[0].text
    # print(data1)
    
    # 날짜 
    data2 = a0.find_elements_by_css_selector('div.title_foot > span.date01')[0].text
    # print(data2)
    
    # 댓글 태그
    a3 = driver.find_elements_by_css_selector('#content > div > div.reply_box > div.reply_top')[0]
    
    # 댓글 수
    a4 = a3.find_elements_by_css_selector('#replyCnt2')[0].text
    # print(a4)
    
    # 댓글 수가 0이면 False 반환
    if not a4:
        print('댓글 없음')
        return False
    
    # 댓글 수가 0이 아니면
    else:
        # 댓글 태그들
        a5 = driver.find_elements_by_css_selector('#spinTopLayerCommentList > li')
        
        # 댓글 수만큼 반복
        for a6 in a5:
            
            # 댓글 내용이 든 태그
            a7 = a6.find_elements_by_css_selector('div.module')[0]
            
            # 작성자
            data33 = a7.find_elements_by_css_selector('div.nickname')[0].text
            # print(a8)
            
            # 작성일 
            data3 = a7.find_elements_by_css_selector('div.createdate')[0].text
            # print(a8)
            
            # 내용
            data4 = a7.find_elements_by_css_selector('div.comment')[0].text
            # print(a9)
            
            comment_dict['제목'].append(data1)
            comment_dict['날짜'].append(data2)
            comment_dict['작성자'].append(data33)
            comment_dict['작성일'].append(data3)
            comment_dict['댓글'].append(data4)
            
            # 댓글달기 답글 수
            a10 = a7.find_elements_by_css_selector('div.reply')[0].text[1]
            # print(a10)
            
            # 답글 유무 확인
            if int(a10) > 0:
                
                # 댓글 보기 클릭
                more_button = a7.find_elements_by_css_selector('div.operation > div.notify > a')[0]
                more_button.send_keys('\n')
                time.sleep(1)
                
                # 대댓글 리스트
                a11 = a7.find_elements_by_css_selector('div.etc > div.spinTopLayerReplyList > ul > li')
                # print(a11)
                
                for a12 in a11:
                    
                    # 작성자
                    data33 = a12.find_elements_by_css_selector('div.module > div.nickname')[0].text
                    print(data33)
                    
                    # 작성일
                    data3 = a12.find_elements_by_css_selector('div.module > div.createdate')[0].text
                    # print(data3)
                    
                    # 댓글 
                    data4 = a12.find_elements_by_css_selector('div.module > div.comment')[0].text
                    # print(data4)                    
                    
                    comment_dict['제목'].append(data1)
                    comment_dict['날짜'].append(data2)
                    comment_dict['작성자'].append(data33)
                    comment_dict['작성일'].append(data3)
                    comment_dict['댓글'].append(data4)
                   
        df1 = pd.DataFrame(comment_dict)
        df1['날짜'] = [date.split('입력')[-1].split('수정')[0].strip() \
                       for date in df1['날짜']]
        df1['날짜'] = pd.to_datetime(df1['날짜'])
        df1['작성일'] = pd.to_datetime(df1['작성일'])
        display(df1)

        FILENAME = 'donga_comment.csv'
        if os.path.exists(FILENAME) == False:
            # 파일이 없을 경우
            df1.to_csv(FILENAME, encoding='utf-8-sig', index=False)
        else:
            # mode='a' : 기존 것 뒤에다 붙여줌
            df1.to_csv(FILENAME, encoding='utf-8-sig', index=False, header=False, mode='a')
 
        return True


In [28]:
def getDongaComment(link_df):
    
    import pandas as pd
    from selenium import webdriver as wd
    from IPython.display import clear_output
    
    num_link = link_df.shape[0]
    # print(link_df.loc[0])

    # 웹 드라이버
    driver = wd.Chrome('./chromedriver.exe')
    idx = 0

    while True :

        driver.implicitly_wait(20)

        clear_output(wait=True)

        driver.get(link_df.loc[idx].values[0])
        driver.refresh()
        clickMore(driver)

        print(f'{idx}번째 기사 댓글 수집 중')

        chk = getComment(driver)

        # 기사에 댓글 데이터 정상 수집
        if chk:
            print(f'{idx}번째 기사 댓글 정상 수집 완료')
        # 기사에 댓글 데이터가 없으면 다음 페이지로 
        else:
            print(f'{idx}번째 기사 댓글 없음, 다음 기사로~')

        idx = idx + 1

        if idx == num_link :
            break

    print('수집 테스트 완료')

In [29]:
# import pandas as pd

# df = pd.read_csv('../동아일보_link.csv')
# df = df[:1]

# df_comments = getDongaComment(df)
# df_comments


0번째 기사 댓글 수집 중
삼기노


,제목,날짜,작성자,작성자링크,작성일,댓글
0,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,권일상,https://www.donga.com/news/article/all/2021040...,2021-04-10 11:33:15,2=제발 착하고 바르고 열심(熱心)히 사는 國民 잘살도록 지능(知能)과 긍휼(矜恤)...
1,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,권일상,https://www.donga.com/news/article/all/2021040...,2021-04-10 11:32:43,1=♥초선 의원(初選 議員)님들 경천애인(敬天愛人) 우국지심(憂國之心)을 고대(苦待...
2,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,hey9142,https://www.donga.com/news/article/all/2021040...,2021-04-09 22:14:27,선거도 허벌라게 깨지고 국정은 꼬일데로 꼬여가니\n재이니 이자식 또 몽니 부리것네\...
3,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,치술령,https://www.donga.com/news/article/all/2021040...,2021-04-09 22:02:58,땅을보고 싹을 틔워라.\n그긴 기름진 옥토가 아니야\n온갖 오물이 범벅된 지하를 못...
4,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,밤꽃필때,https://www.donga.com/news/article/all/2021040...,2021-04-09 20:45:03,최소한 긴 안목으로는 공천않는게 답이었지. 그걸 못했으니 이제 그런 소리 아무 쓸모...
5,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,가자고,https://www.donga.com/news/article/all/2021040...,2021-04-09 19:34:21,짝짝짝! 고뇌에 찬 국민들의 비명 소리에도 눈감고 거수기 노릇하느라 수고 많았어요....
6,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,keukan,https://www.donga.com/news/article/all/2021040...,2021-04-09 18:43:47,시도는 좋은데.......\n초심으로 돌아간다고...???? 초심이 무엇인데요......
7,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,소사벌친구,https://www.donga.com/news/article/all/2021040...,2021-04-09 18:41:32,공천 안하고 하고가 중요한것이 아니라 멀 잘못했는지 아는가 모르는가가 문제다.\n\...
8,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,bljssh,https://www.donga.com/news/article/all/2021040...,2021-04-09 18:17:25,"이번에도 작년 국회의원선거 개표에서처럼 부정을 저질러 볼려고 했는데, 여론조사에서 ..."
9,"민주당 초선들 “보궐선거 공천 말았어야, 어느새 기득권 정당”",2021-04-09 16:08:00,기타맨,https://www.donga.com/news/article/all/2021040...,2021-04-09 18:13:57,에라이 주사 공산개들아 그소리 선거때마다 듣든소리다 혁신 ㅋㅋㅋ며칠지나면 멜롱 이지...


0번째 기사 댓글 정상 수집 완료
수집 테스트 완료


In [ ]:
df